In [35]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch, helpers
import requests
from IPython.display import JSON
import numpy as np
from fuzzywuzzy import fuzz
from joblib import Parallel, delayed
import pickle

In [2]:
scopus = list(MongoClient()["scopus_colombia"]["stage"].find({'DOI': {"$ne": np.nan}},{"_id":0,'Title':1,'Year':1, 'Source title':1,'Authors':1,"DOI":1}))
openalex = list(MongoClient()["colombia_udea"]["works"].find({},{"_id":0,"titles.title":1,"external_ids":1, "year_published":1, "authors.full_name":1, "source.names":1}))

In [3]:
#remove non doi ids
for i in range(len(openalex)):
    rec = openalex[i]
    for j in rec["external_ids"]:
        if j["source"] == "doi":
            openalex[i]["external_ids"] = j
            continue

In [4]:
openalex[0]

{'titles': [{'title': 'New records of Coronatella (Crustacea, Branchiopoda, Chydoridae) from Colombia with the first report of Coronatella undata and of the male of Coronatella monacantha'}],
 'external_ids': {'source': 'doi', 'id': '10.1590/1519-6984.254487'},
 'year_published': 2024,
 'authors': [{'full_name': 'Juan M. Fuentes-Reinés'},
  {'full_name': 'Pedro Eslava'},
  {'full_name': 'Lourdes M. A. Elmoor-Loureiro'},
  {'full_name': 'F. D. Rocha-S'},
  {'full_name': 'Deivis Suárez Rivero'}],
 'source': {'names': [{'lang': 'en', 'name': 'Brazilian Journal of Biology'}]}}

In [13]:
%%time
p = []
n = []
counter = 0
size=3000
p_dois = []
#positives
for i in scopus:
    
    for j in openalex:
        if i["DOI"] ==  j['external_ids']["id"]:
            p.append((i,j))
            p_dois.append(j['external_ids']["id"])
            counter+=1
            if counter%10 == 0:
                print(counter)
            continue
            
        if len(p) == size:
             break
    if len(p) == size:
         break

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222

In [48]:
%%time
#negatives
n=[]
counter=0
for i in scopus:
    for j in openalex:
        if i["DOI"] != j['external_ids']["id"]:
            if i["DOI"] not in p_dois and j['external_ids']["id"] not in p_dois:
                n.append((i,j))
                counter+=1
                if counter == size:
                    print(counter)
                    break
            if counter == size:
                break
            continue
    if counter == size:
        break


3000
CPU times: user 87.9 ms, sys: 5 µs, total: 87.9 ms
Wall time: 86.9 ms


In [44]:
len(p)

3000

In [45]:
len(n)

3000

In [34]:
p[0]

({'Authors': 'Jaramillo-Ayerbe F., Berrío-Muñoz J.',
  'Title': 'Ivermectin for crusted Norwegian scabies induced by use of topical steroids',
  'Year': 1998,
  'Source title': 'Archives of Dermatology',
  'DOI': '10.1001/archderm.134.2.143'},
 {'titles': [{'title': 'Ivermectin for Crusted Norwegian Scabies Induced by Use of Topical Steroids'},
   {'title': 'Ivermectin for crusted Norwegian scabies induced by use of topical steroids'}],
  'external_ids': {'source': 'doi', 'id': '10.1001/archderm.134.2.143'},
  'year_published': 1998,
  'authors': [{'full_name': 'Felipe Jaramillo-Ayerbe'},
   {'full_name': 'Joaquín Berrío-Muñoz'}],
  'source': {'names': [{'name': 'Archives of Dermatology', 'lang': 'en'}]}})

In [36]:
n[0]

({'Authors': 'Flórez G.A.D., Mombello E.E., Voss S.',
  'Title': 'Novel technique for the calculation of eddy current losses and Lorentz forces in foil winding transformers',
  'Year': 2017,
  'Source title': 'International Journal of Applied Electromagnetics and Mechanics',
  'DOI': '10..3233/JAE-160144'},
 {'titles': [{'title': 'New records of Coronatella (Crustacea, Branchiopoda, Chydoridae) from Colombia with the first report of Coronatella undata and of the male of Coronatella monacantha'}],
  'external_ids': {'source': 'doi', 'id': '10.1590/1519-6984.254487'},
  'year_published': 2024,
  'authors': [{'full_name': 'Juan M. Fuentes-Reinés'},
   {'full_name': 'Pedro Eslava'},
   {'full_name': 'Lourdes M. A. Elmoor-Loureiro'},
   {'full_name': 'F. D. Rocha-S'},
   {'full_name': 'Deivis Suárez Rivero'}],
  'source': {'names': [{'lang': 'en',
     'name': 'Brazilian Journal of Biology'}]}})

In [46]:
with open('dataset.pkl', 'wb') as f:
    pickle.dump({"p":p,"n":n}, f)